In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from PIL import Image

In [3]:
# Data loading and preprocessing
class CustomDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.classes, self.class_to_idx = self._find_classes(self.directory)
        self.samples = self.make_dataset()

    def _find_classes(self, dir):
        classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        classes.sort()
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

    def make_dataset(self):
        instances = []
        for target_class in sorted(self.class_to_idx.keys()):
            class_index = self.class_to_idx[target_class]
            target_dir = os.path.join(self.directory, target_class)
            for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root, fname)
                    if path.endswith('.jpg'):
                        item = path, class_index
                        instances.append(item)
        return instances

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, target

In [4]:
# Data augmentation and preprocessing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Splitting dataset
def create_data_loaders(directory, batch_size=64, num_workers=8, pin_memory=True):
    dataset = CustomDataset(directory, transform=None)
    train_idx, test_idx = train_test_split(np.arange(len(dataset)), test_size=0.2, random_state=42)

    train_sampler = WeightedRandomSampler([1]*len(train_idx), len(train_idx))
    test_sampler = torch.utils.data.SubsetRandomSampler(test_idx)

    train_dataset = CustomDataset(directory, transform=data_transforms['train'])
    test_dataset = CustomDataset(directory, transform=data_transforms['test'])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler,
                              num_workers=num_workers, pin_memory=pin_memory)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, sampler=test_sampler,
                             num_workers=num_workers, pin_memory=pin_memory)
    return train_loader, test_loader

In [5]:
# Model definition
class ModifiedResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet50, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.base_model(x)




In [9]:
# Training and evaluation
def train_and_evaluate(train_loader, test_loader, device, num_epochs=10, learning_rate=0.001):
    model = ModifiedResNet50(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()

    start_time = time.time()

    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if i % 20 == 0:
                print(f'Epoch {epoch+1}, Step {i}, Loss: {loss.item()}')

    total_training_time = time.time() - start_time
    print(f'Total training time: {total_training_time:.2f} seconds')

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy on the test set: {100 * correct / total:.2f}%')

In [10]:
if __name__ == "__main__":
    train_loader, test_loader = create_data_loaders('/content/drive/MyDrive/JPEGImages')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_and_evaluate(train_loader, test_loader, device)

Epoch 1, Step 0, Loss: 4.004199504852295
Epoch 1, Step 20, Loss: 2.23380708694458
Epoch 1, Step 40, Loss: 2.064950704574585
Epoch 1, Step 60, Loss: 1.6351743936538696
Epoch 1, Step 80, Loss: 1.4955179691314697
Epoch 1, Step 100, Loss: 1.6246459484100342
Epoch 1, Step 120, Loss: 1.636879801750183
Epoch 2, Step 0, Loss: 1.0605998039245605
Epoch 2, Step 20, Loss: 0.9838219881057739
Epoch 2, Step 40, Loss: 1.2847704887390137
Epoch 2, Step 60, Loss: 1.2749271392822266
Epoch 2, Step 80, Loss: 1.4498692750930786
Epoch 2, Step 100, Loss: 1.081357479095459
Epoch 2, Step 120, Loss: 1.0814461708068848
Epoch 3, Step 0, Loss: 1.1199069023132324
Epoch 3, Step 20, Loss: 0.8310339450836182
Epoch 3, Step 40, Loss: 0.9811151623725891
Epoch 3, Step 60, Loss: 0.9070419073104858
Epoch 3, Step 80, Loss: 0.9649494290351868
Epoch 3, Step 100, Loss: 0.7771704196929932
Epoch 3, Step 120, Loss: 0.8884932994842529
Epoch 4, Step 0, Loss: 0.8512610793113708
Epoch 4, Step 20, Loss: 0.8313724994659424
Epoch 4, Step 4